# Log 

* Written by YongGi Hong/ email : yghong@medicalip.com
* Written date: 20230207

---
## Code desc
* 3D nnUnet whole body segmentation
* Bone mask extract
---

# 1. Package Import

In [9]:
# Path
import os, glob, shutil

# data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# medical image
import nibabel as nib
import dicom2nifti.settings as settings
settings.disable_validate_slice_increment()
from dicom2nifti.exceptions import ConversionValidationError, ConversionError
import dicom2nifti
import pydicom
from pydicom import read_file

# etc
from tqdm import tqdm
import ipywidgets as widgets
from IPython.display import clear_output
from datetime import date
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['nnUNet_raw_data_base'] = './utils/nnUNet_raw_data_base'
os.environ['RESULTS_FOLDER'] = './utils/nnUNet/RESULTS_FOLDER/'
os.environ['nnUNet_preprocessed'] = './utils/nnUNet_preprocessed/'
os.environ['MKL_THREADING_LAYER'] = 'GNU' 

# 2. Variables

In [12]:
# CT_PATH는 3D CT 이미지가 저장된 경로, Mask_PATH는 3D Mask 이미지가 저장되는 경로 
CT_PATH = './data/3D_CT/'
Mask_PATH = './data/3D_Mask/'
Lung_PATH = './data/3D_Lung/'

try :
    os.mkdir(Mask_PATH)
    os.mkdir(Lung_PATH)
except:
    pass


In [13]:
data_paths = glob.glob(CT_PATH + '*.nii')
data_paths.sort()

for data_path in data_paths:
    nii = nib.load(data_path)
    affine = nii.affine
    header = nii.header 
    arr = nii.get_fdata()
    nii = nib.Nifti1Image(arr, affine, header)
    nib.save(nii, data_path[:-4]+'_0000.nii.gz')

# 3. Whole body Mask & Lung Mask


In [14]:
# 3D nnUNet 모델을 사용
!nnUNet_predict -i {CT_PATH} -o {Mask_PATH} -t Task777_class7_segmentation  -m 3d_fullres -f 0 -chk model_best --disable_tta
!nnUNet_predict -i {CT_PATH} -o {Lung_PATH} -t Task595_effusion  -m 3d_fullres -f 0 -chk model_best --disable_tta
# 이름을 원래대로 돌려놓음
data_paths = glob.glob(CT_PATH + '*.nii.gz')
data_paths.sort()
for data_path in data_paths:
    os.remove(data_path)

no separate z, order 1
before: {'spacing': array([0.65625006, 0.65625006, 0.65625006]), 'spacing_transposed': array([0.65625006, 0.65625006, 0.65625006]), 'data.shape (data is transposed)': (1, 557, 512, 512)} 
after:  {'spacing': array([1.        , 0.62695318, 0.62695318]), 'data.shape (data is resampled)': (1, 366, 536, 536)} 

(1, 366, 536, 536)
This worker has ended successfully, no errors to report
predicting ./data/3D_Lung/07667017_20170204.nii.gz
debug: mirroring False mirror_axes (0, 1, 2)
step_size: 0.5
do mirror: False
data shape: (1, 366, 536, 536)
patch size: [ 96 160 160]
steps (x, y, and z): [[0, 45, 90, 135, 180, 225, 270], [0, 75, 150, 226, 301, 376], [0, 75, 150, 226, 301, 376]]
number of tiles: 252
computing Gaussian
prediction done
inference done. Now waiting for the segmentation export to finish...
force_separate_z: None interpolation order: 1
separate z: False lowres axis None
no separate z, order 1
WARNING! Cannot run postprocessing because the postprocessing file

# 4. Extract Mask

    1. Skin
    2. Bone
    3. Muscle
    4. AVF
    5. OF(SF)
    6. IO
    7. BSC(CNS)

In [15]:
mask_list = [ "Skin", "Bone", "Muscle", "AVF", "OF", "IO", "BSC"]
Mask_PATH = './data/3D_Mask/'
Lung_PATH = './data/3D_Lung/'
CT_PATH = './data/3D_CT/'

In [ ]:
def resize_keep_ratio(img, target_size):
	old_size = img.size  # old_size[0] is in (width, height) format

	ratio = float(target_size) / max(old_size)
	new_size = tuple([int(x * ratio) for x in old_size])

	# im.thumbnail(new_size, Image.ANTIALIAS)
	try:
		im = img.resize(new_size, Image.LANCZOS)
	except:
		im = img.resize(new_size, Image.NEAREST)
	return im


def pad_image(img, target_size, pad_value=-1024):
	old_size = img.size
	pad_size_w = (target_size - old_size[0]) / 2
	pad_size_h = (target_size - old_size[1]) / 2

	if pad_size_w % 2 == 0:
		wl, wr = int(pad_size_w), int(pad_size_w)
	else:
		wl = ceil(pad_size_w)
		wr = floor(pad_size_w)

	if pad_size_h % 2 == 0:
		ht, hb = int(pad_size_h), int(pad_size_h)
	else:
		ht = ceil(pad_size_h)
		hb = floor(pad_size_h)
		
	return transforms.Compose(
		[
			transforms.Pad((wl, ht, wr, hb), fill=pad_value),
			# transforms.ToTensor(),
		]
	)


def save_nii(img, path, header=None, resize=False):
    """_summary_

    Args:
        img (_type_): _description_
        path (List ): path[0] = Base_dir, path[0] = save_dir, path[2] = file_name
        header (nib header): nifiti header information
        resize (bool, optional): _description_. Defaults to False.
    """
    
	nii = nib.Nifti1Image(np.transpose(img.astype(np.int16), axes=[1, 0]), affine=None, header=header)
	nib.save(nii, os.path.join(path[0], path[1] ))

	if resize:
		resize = Image.fromarray(img)
		resize = resize_keep_ratio(resize, 2048)
		img_pad = pad_image(resize, 2048)
		resize = np.array(img_pad(resize))

		nii = nib.Nifti1Image(np.transpose(resize.astype(np.int16), axes=[1, 0]), affine=None)
	    nib.save(nii, os.path.join(path[0], path[1] ))






def extract_mask(target_dir, mask_dir, save_dir,extract_idx = 1, resize = True):
    
    target_paths = glob.glob(target_dir + '*.nii')
    mask_paths = glob.glob(mask_dir + '*.nii.gz')
    target_paths.sort()
    mask_paths.sort()
    
    for target_path, mask_path in zip(target_paths, mask_paths):
        file_name = os.path.basename(target_path)
        
        target_nii = nib.load(target_path)
        mask_nii = nib.load(mask_path)
        target_header = target_nii.header
        mask_header = mask_nii.header
        target_arr = np.transpose(np.array(target_nii.get_fdata()), axes=[2, 1, 0])
        mask_arr   = np.transpose(np.array(mask_nii.get_fdata()),   axes=[2, 1, 0])
        
        target_arr = np.where(target_arr < -1024, -1024, target_arr)
	    coronal_projection3d_target_arr = np.transpose(target_arr, axes=[0, 2, 1])
    	coronal_projection_target_arr = np.mean(coronal_projection3d_target_arr, axis=2)
	    save_nii(coronal_projection_target_arr, [ save_dir, file_name], header, resize)
        
        
        
        
        mask_arr = mask_arr == extract_idx
        target_arr = target_arr * mask_arr
        target_nii = nib.Nifti1Image(target_arr, target_nii.affine, target_nii.header)
        nib.save(target_nii, save_dir + target_path.split('/')[-1])
    

In [8]:
mask_list = [ "Skin", "Bone", "Muscle", "AVF", "OF(SF)", "IO", "BSC(CNS)", "whole_body"]
Mask3d = glob.glob(Mask_PATH + '*.nii.gz')
Mask3d.sort()
data_paths = glob.glob(CT_PATH + '*.nii.gz')
data_paths.sort()


for extract_mask_number in range(len(mask_list)):


    print(extract_mask_number , mask_list[extract_mask_number])
    exMask_PATH  = f'./data/3D_{mask_list[extract_mask_number]}/'
    exMask_PATH1  = f'./data/2D_{mask_list[extract_mask_number]}/'

    try : 
        os.makedirs(exMask_PATH)
        os.makedirs(exMask_PATH1)
        print(exMask_PATH)
    except:
        pass
    if extract_mask_number == 7:
        for i in range(len(Mask3d)):
            ori =nib.load(data_paths[i])
            ori_array = ori.get_fdata()
            mask = nib.load(Mask3d[i])
            header = ori.header
            affine = ori.affine
            mask = mask.get_fdata()
            mask[mask >= 1] = 1
            mask_image = np.array(np.where(mask==1,ori_array, -1024))
            # mask_image = np.array(np.where(mask==1,mask,0),dtype=np.float64)
            mask = nib.Nifti1Image(mask_image.astype(np.int16), affine, header)
            file_name = Mask3d[i].split('/')[-1]
            save_path = exMask_PATH + file_name
            nib.save(mask, save_path)
        
        
    else:     
        for i in range(len(Mask3d)):
            ori =nib.load(data_paths[i])
            ori_array = ori.get_fdata()
            mask = nib.load(Mask3d[i])
            header = ori.header
            affine = ori.affine
            mask = mask.get_fdata()
            mask[mask != extract_mask_number+1] = 0
            mask[mask == extract_mask_number+1] = 1
            mask_image = np.array(np.where(mask==1,ori_array, -1024))
            # mask_image = np.array(np.where(mask==1,mask,0),dtype=np.float64)
            mask = nib.Nifti1Image(mask_image.astype(np.int16), affine, header)
            file_name = Mask3d[i].split('/')[-1]
            save_path = exMask_PATH + file_name
            nib.save(mask, save_path)


0 Skin
1 Bone
2 Muscle
3 AVF
4 OF(SF)
5 IO
6 BSC(CNS)
7 whole_body
